In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial.distance import cosine

from ase import Atoms, geometry, neighborlist
from ase.io import read, write
from ase.neighborlist import NeighborList
from ase.visualize import view

from scipy import sparse
from scipy.spatial.transform import Rotation

import networkx as nx

import metatensor
from rascaline import SoapPowerSpectrum
from anisoap.representations import EllipsoidalDensityProjection
from anisoap.utils import ClebschGordanReal, cg_combine, standardize_keys

In [ ]:
mols = read("planar_mols.xyz", ":")

NLIST_KWARGS = {
    "skin": 0.3,   # doesn't matter for this application.
    "sorted": False,
    "self_interaction": False,
    "bothways": True
}

In [ ]:
def initiate_tags(frame):
    frame.arrays["corresponding_ellipsoid"] = np.array(len(frame) * [-1])

def build_graph(mol: Atoms, nl=None):
    if nl == None:
        nl = mol.build_neighbor_list(mol)
    G = nx.Graph()
    for i in range(len(mol)):
        atom = mol[i]
        nb_indices, offsets = nl.get_neighbors(i)
        nb_atoms = [mol[a] for a in nb_indices]
        el = [(i, nb) for nb in nb_indices]
        G.add_edges_from(el)
        G.nodes[i]["atom"] = mol[i]
    return G

def get_rings(frame, graph=None):
    if graph == None:
        G = build_graph(frame)
    else:
        G = graph
    rings = nx.cycle_basis(G)
    return rings

def tag_by_ellipsoid(frame):
    rings = get_rings(frame)
    tags = [-1 for _ in range(len(frame))]
    for i in range(len(frame)):
        for j, ring in enumerate(rings):
            if i in ring:
                tags[i] = j
                break
    frame.arrays["corresponding_ellipsoid"] = np.array(tags)

def get_cluster_data(ring)
    moments, axes = atoms.get_moments_of_inertia(vectors=True)
    mass = np.sum([atom.mass for atom in ring_atoms])
    E = np.reshape(moments, (3,1))
    coefs = np.array([[0, 1, 1],
                      [1, 0, 1],
                      [1, 1, 0]]) * mass / 5
    return {"axes" : np.sqrt(np.linalg.solve(coefs, E)), "moments" : moments, "eigenvectors" : axes, "mass" : mass}
    
    
def get_ellipsoids(frame):
    coms = []
    quats = []
    positions = []
    for ell_id in np.unique(frame.arrays["corresponding_ellipsoid"]):
        cluster = frame[(frame.arrays['corresponding_ellipsoid'] == ell_id) & (frame.arrays['numbers'] == 6)]
        dist_vecs, _ = geometry.get_distances(cluster.positions)
        pos_vecs = cluster.positions[0] + dist_vecs[0]
        com = pos_vecs.mean(axis=0)
        data = get_cluster_data(cluster)
        rot = np.asarray(data["eigenvectors"])
        quat = Rotation.from_matrix(rot).as_quat()
        quat = np.roll(quat, 1)
        quats.append(quat)
        positions.append(pos_vecs)
        coms.append(com)
